# Project - ECG Arrhythmia Classification
### Natalia Kolińska, Alicja Smaruj, Dorota Woźna, Kacper Zielak


#### Imports

In [16]:
import pandas as pd

#### Load data
Source: [Kaggle](https://www.kaggle.com/datasets/sadmansakib7/ecg-arrhythmia-classification-dataset?select=Sudden+Cardiac+Death+Holter+Database.csv)

In [17]:
df_1 = pd.read_csv('data/INCART 2-lead Arrhythmia Database.csv')
df_2 = pd.read_csv('data/MIT-BIH Arrhythmia Database.csv')
df_3 = pd.read_csv('data/MIT-BIH Supraventricular Arrhythmia Database.csv')
test_data = pd.read_csv('data/Sudden Cardiac Death Holter Database.csv')

c:\users\alicj\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [18]:
train_data = pd.concat([df_1, df_2, df_3])
train_data.head()

,record,type,0_pre-RR,0_post-RR,0_pPeak,0_tPeak,0_rPeak,0_sPeak,0_qPeak,0_qrs_interval,...,1_qPeak,1_qrs_interval,1_pq_interval,1_qt_interval,1_st_interval,1_qrs_morph0,1_qrs_morph1,1_qrs_morph2,1_qrs_morph3,1_qrs_morph4
0,I01,N,163.0,165.0,0.069610,-0.083281,0.614133,-0.392761,0.047159,15,...,-0.023370,14,3,23,6,-0.023370,-0.011650,0.082608,0.101373,-0.183387
1,I01,N,165.0,166.0,-0.097030,0.597254,-0.078704,-0.078704,-0.137781,3,...,0.081637,15,5,27,7,0.081637,0.102992,0.191225,0.217544,-0.068248
2,I01,N,166.0,102.0,0.109399,0.680528,-0.010649,-0.010649,-0.720620,6,...,-0.148539,33,13,52,6,-0.148539,-0.060620,0.081080,0.204400,0.335172
3,I01,VEB,102.0,231.0,0.176376,0.256431,-0.101098,-0.707525,-0.101098,4,...,0.046898,21,9,34,4,0.046898,0.083728,0.279512,0.526785,0.450969
4,I01,N,231.0,165.0,0.585577,0.607461,-0.083499,-0.083499,-0.167858,3,...,-0.112552,32,5,43,6,-0.112552,0.012989,0.091491,0.134004,0.265232


In [19]:
test_data.head()

,record,type,0_pre-RR,0_post-RR,0_pPeak,0_tPeak,0_rPeak,0_sPeak,0_qPeak,0_qrs_interval,...,1_qPeak,1_qrs_interval,1_pq_interval,1_qt_interval,1_st_interval,1_qrs_morph0,1_qrs_morph1,1_qrs_morph2,1_qrs_morph3,1_qrs_morph4
0,30.0,N,2.0,194.0,0.003261,0.159554,-0.046726,-0.046726,-0.129022,4.0,...,-0.131451,29.0,17.0,95.0,49.0,-0.131451,-0.015809,0.109493,-0.153622,-0.200331
1,30.0,N,194.0,188.0,-0.058462,0.067970,0.285091,-0.272093,-0.062109,25.0,...,-0.048727,25.0,2.0,76.0,49.0,-0.048727,0.034657,0.282264,-0.013981,-0.226871
2,30.0,N,188.0,194.0,-0.012959,-0.089749,0.329466,-0.204659,-0.012971,29.0,...,-0.005256,29.0,5.0,53.0,19.0,-0.005256,0.007639,0.116198,0.271552,-0.114414
3,30.0,N,194.0,198.0,-0.032815,-0.113236,0.398437,-0.178191,-0.068497,36.0,...,-0.069992,35.0,26.0,76.0,15.0,-0.069992,-0.049489,0.030777,0.378075,-0.074389
4,30.0,N,198.0,196.0,-0.047353,0.164835,0.364677,-0.207953,-0.061457,39.0,...,-0.072007,35.0,25.0,115.0,55.0,-0.072007,-0.053251,0.044583,0.313201,-0.157637


#### Explore data

Dependent variable
- N (Normal),
- SVEB (Supraventricular ectopic beat),
- VEB (Ventricular ectopic beat),
- F (Fusion beat),
- Q (Unknown beat).

In [27]:
train_data.type.unique()

array(['N', 'VEB', 'SVEB', 'F', 'Q'], dtype=object)

Check if anything is null

In [28]:
train_data.isnull().any()

type              False
0_pre-RR          False
0_post-RR         False
0_pPeak           False
0_tPeak           False
0_rPeak           False
0_sPeak           False
0_qPeak           False
0_qrs_interval    False
0_pq_interval     False
0_qt_interval     False
0_st_interval     False
0_qrs_morph0      False
0_qrs_morph1      False
0_qrs_morph2      False
0_qrs_morph3      False
0_qrs_morph4      False
1_pre-RR          False
1_post-RR         False
1_pPeak           False
1_tPeak           False
1_rPeak           False
1_sPeak           False
1_qPeak           False
1_qrs_interval    False
1_pq_interval     False
1_qt_interval     False
1_st_interval     False
1_qrs_morph0      False
1_qrs_morph1      False
1_qrs_morph2      False
1_qrs_morph3      False
1_qrs_morph4      False
dtype: bool

In [29]:
test_data.isnull().any()

type              True
0_pre-RR          True
0_post-RR         True
0_pPeak           True
0_tPeak           True
0_rPeak           True
0_sPeak           True
0_qPeak           True
0_qrs_interval    True
0_pq_interval     True
0_qt_interval     True
0_st_interval     True
0_qrs_morph0      True
0_qrs_morph1      True
0_qrs_morph2      True
0_qrs_morph3      True
0_qrs_morph4      True
1_pre-RR          True
1_post-RR         True
1_pPeak           True
1_tPeak           True
1_rPeak           True
1_sPeak           True
1_qPeak           True
1_qrs_interval    True
1_pq_interval     True
1_qt_interval     True
1_st_interval     True
1_qrs_morph0      True
1_qrs_morph1      True
1_qrs_morph2      True
1_qrs_morph3      True
1_qrs_morph4      True
dtype: bool

Check how many percentage are null values in test_data

In [35]:
round(test_data.isnull().sum() * 100 / len(test_data), 2)

type              15.16
0_pre-RR          15.16
0_post-RR         15.16
0_pPeak           15.16
0_tPeak           15.16
0_rPeak           15.16
0_sPeak           15.16
0_qPeak           15.16
0_qrs_interval    15.16
0_pq_interval     15.16
0_qt_interval     15.16
0_st_interval     15.16
0_qrs_morph0      15.16
0_qrs_morph1      15.16
0_qrs_morph2      15.16
0_qrs_morph3      15.16
0_qrs_morph4      15.16
1_pre-RR          34.73
1_post-RR         34.73
1_pPeak           34.73
1_tPeak           34.73
1_rPeak           34.73
1_sPeak           34.73
1_qPeak           34.73
1_qrs_interval    34.73
1_pq_interval     34.73
1_qt_interval     34.73
1_st_interval     34.73
1_qrs_morph0      34.73
1_qrs_morph1      34.73
1_qrs_morph2      34.73
1_qrs_morph3      34.73
1_qrs_morph4      34.73
dtype: float64

Check if outliers influence arrhythmia

In [60]:
print('Q1:', train_data['0_pPeak'].quantile(.25), 'Q2:', train_data['0_pPeak'].quantile(.5), 'Q3:', train_data['0_pPeak'].quantile(.75))
print('Min:', min(train_data['0_pPeak']), 'Max:', max(train_data['0_pPeak']))

Q1: -0.05114678272548937 Q2: 0.0120459091030483 Q3: 0.11941641509086005
Min: -4.208600735 Max: 10.56590393


In [75]:
q3 = train_data['0_pPeak'].quantile(.75)
print('Amount of outliers:', len(train_data.loc[train_data['0_pPeak'] > q3]))
print('Percentage of type in the outlier group')
train_data.loc[train_data['0_pPeak'] > q3].groupby('type').size() * 100/len(train_data.loc[train_data['0_pPeak'] > q3])

Amount of outliers: 115212
Percentage of type in the outlier group


type
F        0.348922
N       75.889664
Q        0.057286
SVEB     4.189668
VEB     19.514460
dtype: float64

In [69]:
test_data.groupby('type').size() * 100/len(test_data)

type
F        0.083341
N       94.805458
Q        0.017065
SVEB     0.623075
VEB      4.471061
dtype: float64

#### Prepare data

Remove 'record' column, because the name of the subject/patient is irrelevant in this study.

In [20]:
train_data = train_data.drop('record', axis=1)
test_data = test_data.drop('record', axis=1)

Remove null observations from test_data

In [38]:
test_data = test_data.dropna()
len(test_data)

251976

#### Model 1 - Natalia

#### Model 2 - Dorota

#### Model 3 - Kacper